## Gathering the data

In [9]:
from tdmclient import ClientAsync
import time
import numpy as np
client = ClientAsync()

node = await client.wait_for_node()
await node.lock()

init_time = time.time()
last_time = init_time
left_speed = []
right_speed = []

v = {
    "motor.left.target": [200],
    "motor.right.target": [200],
    }
await node.set_variables(v)
await client.sleep(0.1)   

while time.time()-init_time<10:
    await node.wait_for_variables({"motor.left.speed"})
    left_speed = np.append(left_speed, node["motor.left.speed"])
    right_speed = np.append(right_speed, node["motor.right.speed"])
    last_time = time.time()
    await client.sleep(0.01)

In [10]:
v = {
    "motor.left.target": [0],
    "motor.right.target": [0],
    }
await node.set_variables(v)

await node.unlock()

## Calculating the variances

In [11]:
left_speed = np.array(left_speed)[50:]
print("We have", str(left_speed.shape), "datapoints for the left motor speed")
right_speed = np.array(right_speed)[50:]
print("We have", str(right_speed.shape), "datapoints for the right motor speed")

left_var = np.var(left_speed) 
print("The variance on the left motor speed is",left_var)
right_var = np.var(right_speed)
print("The variance on the right motor speed is",right_var)

We have (728,) datapoints for the left motor speed
We have (728,) datapoints for the right motor speed
The variance on the left motor speed is 41.32967032967033
The variance on the right motor speed is 70.12343768868494


## Translating to x, y and $\theta$

In [12]:
thymio_speed_to_ms = 19.73913043478261/(50*0.001) # dividing by this gives motor speeds in m/s
R = 0.021 # [m] The radius of the Thymio's wheels
d = 0.095 # [m] The wheelbase of the Thymio
dt = 0.05

# Converting the standard deviations of the motor speeds to m/s
left_sd = np.sqrt(left_var)/thymio_speed_to_ms 
right_sd = np.sqrt(right_var)/thymio_speed_to_ms

# Translating to x, y and theta motor speeds
x_sd = 0.5*R*(left_sd+right_sd)*dt
y_sd = x_sd #Both standard deviations are the same for x and y
theta_sd = (R/d)*(left_sd-right_sd)*dt

process_variances = [x_sd**2, y_sd**2, theta_sd**2]
print("The final process variances on x y and theta are:", process_variances)

The final process variances on x y and theta are: [3.8751605996417765e-10, 3.8751605996417765e-10, 2.9656863710880975e-09]
